# Query & load data: `select & load`

- LaminDB allows querying data based on SQL-derived `select` statements.
- Once data is queried, you can load it into memory using `load`.

To simulate team collaboration, let us login as another user who queries data ingested by user 1.

In [ ]:
!lndb login testuser2

In [ ]:
import lamindb as ln
import lamindb.schema as lns

ln.nb.header()

## Select

LaminDB's {func}`~lamindb.select` statements are based on [SQLModel](https://sqlmodel.tiangolo.com) but offer further simplifications targeted at data scientists.[^sql]

[^sql]: And SQLModel is a shallow wrapper around SQLAlchemy, which provides the canonical way of creating SQL statements from Python.

In [ ]:
stmt = (
    ln.select(lns.DObject)
    .join(lns.DTransform)
    .join(lns.Jupynb)
    .join(lns.User)
    .where(lns.User.handle == "testuser1")
)

To access the query results encoded in `dobjects` (a `SelectStmt`), execute the statemnt with one of

- `.all()`: A list of records.
- `.df()`: A dataframe with each record stored as a row.
- `.one()`: Exactly one record. Will raise an error if there is none.
- `.one_or_none()`: Either one record or `None` if there is no query result.

For example:

In [ ]:
stmt.all()

In [ ]:
stmt.df()

Let us subset to just the parquet files - we know it's exactly a single one. So we can get the record using `.one()`.

In [ ]:
ln.select(lns.DObject).where(lns.DObject.suffix == ".parquet").join(
    lns.DTransform
).join(lns.Jupynb).join(lns.User).where(lns.User.handle == "testuser1").one()

Or subset to files greater than 10kB.

In [ ]:
ln.select(lns.DObject).where(lns.DObject.size > 1e4).join(lns.DTransform).join(
    lns.Jupynb
).join(lns.User).where(lns.User.handle == "testuser1").df()

Or select a notebook based on a substring in the name:

In [ ]:
ln.select(lns.Jupynb).where(lns.Jupynb.name.contains("Ingest")).df()

Or select datasets based on a gene symbol.

In [ ]:
ln.select(lns.DObject).join(lns.wetlab.DObjectBiometa).join(lns.wetlab.Biometa).join(
    lns.bionty.Featureset
).join(lns.bionty.FeaturesetGene).join(lns.bionty.Gene).where(
    lns.bionty.Gene.symbol == "Actg1"
).df()

```{note}

Write an example involving `lookup.readout.single_cell_RNA_sequencing`.

```

## Load

Load data objects into memory via {func}`~lamindb.load`:

In [ ]:
dobject = ln.select(lns.DObject, name="iris").first()

df = ln.load(dobject)

In [ ]:
df.head()

If no in-memory format can be found, `load` returns the filepath:

In [ ]:
dobject = ln.select(lns.DObject).where(lns.DObject.name.contains("paradisi05")).one()

In [ ]:
ln.load(dobject)

## Using `SQLModel` sessions

If needed, you can also use the lower level [SQLModel](https://sqlmodel.tiangolo.com) API via `ln.session()`.